# **Dependancies**

In [1]:
import keras_tuner as kt

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
# from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, PredefinedSplit
from sklearn.ensemble import GradientBoostingRegressor

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from collections import Counter
from scipy.stats import uniform
from scipy.stats import randint


2022-12-04 11:03:14.836731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# **Loading Data**

In [3]:
selected = pd.read_csv("../Data/train_btc_selected_features.csv")
btc = pd.read_csv("../Data/btc_Data.csv")
btc['Date'] = pd.to_datetime(btc['Date'])
btc = btc.set_index("Date")

In [4]:
btcData = btc[selected.columns]
btcData['returns'] = btcData['priceUSD'].pct_change().copy()
Data = btcData.drop(columns=['priceUSD'])
Data = Data[1:]
# divide X and Y
X = Data.iloc[:,0:]
#Y = Data['returns']   # 用returns的话就用这一行，然后把下一行comment掉
Y = btcData['priceUSD'].shift(-30)[1:] # 反之亦然
# Split into three data sets
X_train = X['2016-01-01':'2019-12-31']
X_val = X['2020-01-01':'2021-05-31']
X_test = X['2021-06-01':'2023-01-01']

Y_train = Y['2016-01-01':'2019-12-31']
Y_val = Y['2020-01-01':'2021-05-31']
Y_test = Y['2021-06-01':'2023-01-01']

/tmp/ipykernel_2663409/3854812091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btcData['returns'] = btcData['priceUSD'].pct_change().copy()


In [5]:
estimators=[]
estimators.append(['mixmax',MinMaxScaler()])
estimators.append(['robust',RobustScaler()])
scale=Pipeline(estimators,verbose=True)
scale.fit(X_train)
X_train=scale.transform(X_train)
X_test=scale.transform(X_test)
X_val = scale.transform(X_val)

[Pipeline] ............ (step 1 of 2) Processing mixmax, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing robust, total=   0.0s


In [6]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))
X_val=np.reshape(X_val,(X_val.shape[0],1,X_val.shape[1]))
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))
Y_train=Y_train.values
Y_train=np.reshape(Y_train, (Y_train.shape[0],1,1))
Y_val=Y_val.values
Y_val=np.reshape(Y_val, (Y_val.shape[0],1,1))
Y_test=Y_test.values
Y_test=np.reshape(Y_test, (Y_test.shape[0],1,1))

In [7]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [8]:
adam=Adam(lr=lr_schedule(0),amsgrad=True)


Learning rate:  0.001


/home/spectre/anaconda3/envs/tensorplustorch/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
def build_model(hp):
    model = Sequential()

    #first layer
    model.add(Bidirectional(LSTM(hp.Int('input_unit1',min_value=32,max_value=512,step=32), return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(Bidirectional(LSTM(hp.Int(f'input_unit{i+1}',min_value=32,max_value=512,step=32), return_sequences=True, activation='relu')))
    #dropout layer
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    # Dense layer
    model.add(Dense(1, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
    model.compile(loss="logcosh", optimizer='adam', metrics=['mae'])
    return model

In [ ]:
tuner= kt.RandomSearch(
        build_model,
        objective='val_mse',
        max_trials=10,
        executions_per_trial=1,
        directory='btc_tune',
        project_name='LSTM_TUNE'
        )

tuner.search(
        x=X_train,
        y=Y_train,
        epochs=5000,
        batch_size=32,
        validation_data=(X_val,Y_val),
)

INFO:tensorflow:Reloading Oracle from existing project btc_tune/LSTM_TUNE/oracle.json


2022-12-04 11:03:16.723121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 11:03:16.723985: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
384               |?                 |input_unit1
1                 |?                 |n_layers
0.3               |?                 |Dropout_rate
sigmoid           |?                 |dense_activation

Epoch 1/5000
46/46 [==============================] - 5s 57ms/step - loss: 5026.0776 - mae: 5026.7710 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 2/5000
46/46 [==============================] - 2s 42ms/step - loss: 5025.9746 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3/5000
46/46 [==============================] - 2s 43ms/step - loss: 5025.9751 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 4/5000
46/46 [==============================] - 2s 39ms/step - loss: 5025.9746 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 5/5000
46/46 [==============================] - 2s 45ms/step - loss: 5025.9751 - mae: 5026.6680 - val_loss: 23407.9863 - v

46/46 [==============================] - 2s 41ms/step - loss: 5025.9741 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 55/5000
46/46 [==============================] - 2s 42ms/step - loss: 5025.9731 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 56/5000
46/46 [==============================] - 2s 47ms/step - loss: 5025.9751 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 57/5000
46/46 [==============================] - 2s 37ms/step - loss: 5025.9756 - mae: 5026.6689 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 58/5000
46/46 [==============================] - 2s 46ms/step - loss: 5025.9741 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 59/5000
46/46 [==============================] - 2s 44ms/step - loss: 5025.9746 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 60/5000
46/46 [==============================] - 2s 44ms/step - loss: 5025.9741 - mae: 5026.6680 - val_loss: 23407.9863 

46/46 [==============================] - 2s 42ms/step - loss: 5025.9746 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 110/5000
46/46 [==============================] - 2s 45ms/step - loss: 5025.9741 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 111/5000
46/46 [==============================] - 3s 55ms/step - loss: 5025.9751 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 112/5000
46/46 [==============================] - 2s 49ms/step - loss: 5025.9756 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 113/5000
46/46 [==============================] - 2s 46ms/step - loss: 5025.9741 - mae: 5026.6689 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 114/5000
46/46 [==============================] - 2s 44ms/step - loss: 5025.9746 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 115/5000
46/46 [==============================] - 2s 52ms/step - loss: 5025.9761 - mae: 5026.6675 - val_loss: 23407

...

46/46 [==============================] - 2s 45ms/step - loss: 5025.9746 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 2952/5000
46/46 [==============================] - 2s 42ms/step - loss: 5025.9756 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 2953/5000
46/46 [==============================] - 2s 38ms/step - loss: 5025.9746 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 2954/5000
46/46 [==============================] - 2s 41ms/step - loss: 5025.9741 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 2955/5000
46/46 [==============================] - 2s 46ms/step - loss: 5025.9746 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 2956/5000
46/46 [==============================] - 2s 40ms/step - loss: 5025.9741 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 2957/5000
46/46 [==============================] - 2s 47ms/step - loss: 5025.9746 - mae: 5026.6675 - val_loss:

Epoch 3006/5000
46/46 [==============================] - 2s 51ms/step - loss: 5025.9756 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3007/5000
46/46 [==============================] - 2s 53ms/step - loss: 5025.9751 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3008/5000
46/46 [==============================] - 2s 53ms/step - loss: 5025.9751 - mae: 5026.6670 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3009/5000
46/46 [==============================] - 2s 45ms/step - loss: 5025.9746 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3010/5000
46/46 [==============================] - 2s 47ms/step - loss: 5025.9756 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3011/5000
46/46 [==============================] - 2s 47ms/step - loss: 5025.9741 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3012/5000
46/46 [==============================] - 2s 47ms/step - loss: 5025.9756 - mae: 5026.

46/46 [==============================] - 2s 47ms/step - loss: 5025.9746 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3061/5000
46/46 [==============================] - 2s 50ms/step - loss: 5025.9741 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3062/5000
46/46 [==============================] - 3s 56ms/step - loss: 5025.9741 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3063/5000
46/46 [==============================] - 2s 45ms/step - loss: 5025.9746 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3064/5000
46/46 [==============================] - 2s 49ms/step - loss: 5025.9756 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3065/5000
46/46 [==============================] - 2s 45ms/step - loss: 5025.9746 - mae: 5026.6689 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3066/5000
46/46 [==============================] - 2s 41ms/step - loss: 5025.9741 - mae: 5026.6685 - val_loss:

Epoch 3115/5000
46/46 [==============================] - 2s 44ms/step - loss: 5025.9746 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3116/5000
46/46 [==============================] - 2s 44ms/step - loss: 5025.9746 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3117/5000
46/46 [==============================] - 2s 44ms/step - loss: 5025.9741 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3118/5000
46/46 [==============================] - 2s 40ms/step - loss: 5025.9751 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3119/5000
46/46 [==============================] - 2s 43ms/step - loss: 5025.9736 - mae: 5026.6689 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3120/5000
46/46 [==============================] - 2s 40ms/step - loss: 5025.9741 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3121/5000
46/46 [==============================] - 2s 49ms/step - loss: 5025.9741 - mae: 5026.

46/46 [==============================] - 2s 41ms/step - loss: 5025.9751 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3170/5000
46/46 [==============================] - 2s 41ms/step - loss: 5025.9741 - mae: 5026.6685 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3171/5000
46/46 [==============================] - 2s 44ms/step - loss: 5025.9746 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3172/5000
46/46 [==============================] - 2s 54ms/step - loss: 5025.9751 - mae: 5026.6675 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3173/5000
46/46 [==============================] - 2s 42ms/step - loss: 5025.9736 - mae: 5026.6680 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3174/5000
46/46 [==============================] - 2s 41ms/step - loss: 5025.9746 - mae: 5026.6689 - val_loss: 23407.9863 - val_mae: 23408.6797
Epoch 3175/5000
46/46 [==============================] - 2s 40ms/step - loss: 5025.9751 - mae: 5026.6685 - val_loss:

In [ ]:
# Get the best hyperparameters.
best_hp = tuner.get_best_hyperparameters()[0]
# Build the model with the best hp.
regressor = build_model(best_hp)
# Fit with the entire dataset.
X_val = np.concatenate((X_train, X_val))
Y_val = np.concatenate((Y_train, Y_val))


In [ ]:
earlyStopping = EarlyStopping(monitor='loss', patience=1000, verbose=1, mode='auto')

In [ ]:
regressor.fit(x=X_all, y=Y_all, epochs=5000, batch_size=32, use_multiprocessing=True, callbacks=[earlyStopping])

In [ ]:

y_pred=regressor.predict(X_test)

In [ ]:
# for check
Y_train_pred=regressor.predict(X_train)
r2_score(Y_train, Y_train_pred) #training score

In [ ]:
r2=r2_score(Y_test[:-30],y_pred[:-30]) #score/ r^2
print(f'r2:{r2}')

In [ ]:
# r2_oos
def r2_oos(ret, pred):
    sum_of_sq_res = np.nansum(np.power((ret-pred), 2))
    sum_of_sq_total = np.nansum(np.power(ret, 2))
    
    return 1-sum_of_sq_res/sum_of_sq_total

In [ ]:
mae=mean_absolute_error(Y_test[:-30],y_pred[:-30]) #mae
print(f'mae:{mae}')

rmse=np.sqrt(mean_squared_error(Y_test[:-30],y_pred[:-30])) #rmse
print(f'rmse:{rmse}')

mape=mean_absolute_percentage_error(Y_test[:-30],y_pred[:-30]) #mape
print(f'mape:{mape}')

In [ ]:
y_pred = y_pred.ravel()

In [ ]:
r2_oos = r2_oos(Y_test[:-30], y_pred[:-30])
print(f'r2_oos:{r2_oos}')

-----------------------------

In [ ]:
pre_df = pd.DataFrame(zip(Y_test,y_pred),columns=['Y_test','y_pred'])
pre_df.index = Y_test.index
pre_df

In [ ]:
pre_df['pred_returns'] = pre_df['y_pred'].pct_change()

In [ ]:
pre_df

In [ ]:
pd.DataFrame(zip(Y_test,y_pred),columns=['Y_test','y_pred']).plot()

In [ ]:
pre_df.to_csv("../result/LSTM/btc_lstm.csv")

In [ ]:

!kdeconnect-cli -n TAS-AN00 --ping-msg 'Script complete!'